In [110]:
import json
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

In [111]:
# === Configuración ===
json_file_path = "arxiv-part-10.json"
batch_size = 2000
max_workers = 4  # Ajusta según tu CPU y MongoDB

In [112]:
# === Conectar a MongoDB ===
client = MongoClient("mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client["arxiv_db"]
collection = db["articles"]
archivos = ["arxiv-part-00.json", "arxiv-part-01.json", "arxiv-part-02.json", "arxiv-part-03.json", "arxiv-part-04.json", "arxiv-part-05.json", "arxiv-part-06.json", "arxiv-part-07.json", "arxiv-part-08.json", "arxiv-part-09.json", "arxiv-part-10.json"]

In [113]:
# === Función para insertar lote ===
def insert_batch(batch):
    try:
        collection.insert_many(batch, ordered=False)
    except Exception as e:
        print("Error al insertar batch:", e)

In [114]:
# === Contar líneas para tqdm ===
def count_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

total_lines = count_lines(json_file_path)

In [ ]:
# === Lectura + carga paralela ===
for i in archivos:
    json_file_path = i
    # Usamos total_lines para la barra de carga, asegúrate que esté definida para cada archivo
    with open(json_file_path, 'r', encoding='utf-8') as f, \
         tqdm(total=total_lines, desc=f"Cargando {json_file_path.split('/')[-1]}") as pbar_loading:
        batch = []
        futures = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for line in f:
                try:
                    record = json.loads(line)
                    # 👉 Agregar campo pdf_source
                    record["pdf_source"] = f"https://arxiv.org/pdf/{record['id']}"
                    batch.append(record)
                    if len(batch) >= batch_size:
                        future = executor.submit(insert_batch, batch)
                        futures.append(future)
                        batch = []
                except json.JSONDecodeError:
                    continue
                pbar_loading.update(1) # Actualiza la barra de carga

            # Último batch
            if batch:
                futures.append(executor.submit(insert_batch, batch))
                pbar_loading.update(len(batch)) # Actualiza con los elementos restantes
        
    # === Esperar a que terminen todas las cargas ===
    for future in futures:
        future.result() # Espera a que los futuros se completen y maneja excepciones si es necesario
    
    print(f"\n✅ Carga paralela completa para {json_file_path}. ") # Salto de línea para la siguiente barra
    
    print("El script va a pausar por 60 segundos...")
    for _ in tqdm(range(60), desc="Pausa", unit="seg"):
        time.sleep(1) 
    print("¡El script continuó después de la pausa!")

Cargando arxiv-part-00.json: 276984it [00:32, 8431.38it/s]                                                                                                            



✅ Carga paralela completa para arxiv-part-00.json. 
El script va a pausar por 70 segundos...


Pausa: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [01:00<00:00,  1.00s/seg]


¡El script continuó después de la pausa!


Cargando arxiv-part-01.json: 276984it [00:36, 7664.18it/s]                                                                                                            


...

¡El script continuó después de la pausa!


Cargando arxiv-part-10.json: 12it [00:00, 58.54it/s]                                                                                                                  



✅ Carga paralela completa para arxiv-part-10.json. 
El script va a pausar por 70 segundos...


Pausa: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [01:00<00:00,  1.00s/seg]

¡El script continuó después de la pausa!


In [13]:
'''
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27018/")
db = client["arxiv_db"]
collection = db["articles"]

# ⚠️ Eliminar todos los documentos
collection.delete_many({})
print("🗑️ Todos los documentos eliminados.")
'''

🗑️ Todos los documentos eliminados.
